In [3]:
# import libraries
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

In [4]:
df_games = pd.read_parquet('../data/df_games.parquet')
df_reviews = pd.read_parquet('../data/processed/df_reviews.parquet')
df_items = pd.read_parquet('../data/processed/df_items.parquet')


## def developer(developer: str)
Number of items and percentage of free content per year by developer company

In [5]:
def developer(developer):
    # Filter the dataframe by developer
    df_dev = df_games[df_games['developer'] == developer]
    
    # Calculate the quantity of items released by year
    items_by_year = df_dev.groupby(df_dev['release_year'])['id'].count()
    
    # Calculate the percentage of free content
    free_content_by_year = (df_dev[df_dev['price'] == 0].groupby(df_dev['release_year'])['id'].count() / items_by_year * 100).fillna(0)
    
    # Create a list of dictionaries with the results
    result = [{'Year': int(year), 'Items Released': int(count), '% of Free Content': percent} for year, count, percent in zip(items_by_year.index, items_by_year.values, free_content_by_year.values)]
    
    # Convert the list to a dictionary
    output = {f'Year: {item["Year"]}': {'Items Released': item['Items Released'], '% of Free Content': item['% of Free Content']} for item in result}
    
    return output

In [6]:
developer('Valve')

{'Year: 1998': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 1999': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2000': {'Items Released': 2, '% of Free Content': 0.0},
 'Year: 2001': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2003': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2004': {'Items Released': 5, '% of Free Content': 0.0},
 'Year: 2006': {'Items Released': 2, '% of Free Content': 0.0},
 'Year: 2007': {'Items Released': 4, '% of Free Content': 50.0},
 'Year: 2008': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2009': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2010': {'Items Released': 2, '% of Free Content': 50.0},
 'Year: 2011': {'Items Released': 2, '% of Free Content': 0.0},
 'Year: 2012': {'Items Released': 1, '% of Free Content': 0.0},
 'Year: 2016': {'Items Released': 1, '% of Free Content': 100.0},
 'Year: 2017': {'Items Released': 3, '% of Free Content': 0.0}}

## def best_developer_year(year: int)
returns top 3 most recommended developers in a given year. [{First place: X},{Second place: Y},{Third place: Z}]

In [8]:
df_recs = pd.read_csv('../data/processed/df_recommendation.csv')
df_recs.head()

,user_id,rating,item_name
0,76561197970982479,5,Killing Floor
1,76561197970982479,5,Zeno Clash
2,76561197970982479,5,Metro 2033
3,js41637,5,Barbie™ Dreamhouse Party™
4,js41637,5,Euro Truck Simulator 2


In [9]:
df_developer = df_reviews.merge(df_games, left_on='item_id', right_on='id')
df_developer.head()

,user_id,posted,item_id,recommend,review,sentiment_category,genres,title,release_year,price,id,developer
0,76561197970982479,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...,2,Action,Killing Floor,2009,19.99,1250.0,Tripwire Interactive
1,76561197970982479,2011-07-15,22200.0,True,It's unique and worth a playthrough.,2,Action,Zeno Clash,2009,9.99,22200.0,ACE Team
2,76561197970982479,2011-07-15,22200.0,True,It's unique and worth a playthrough.,2,Indie,Zeno Clash,2009,9.99,22200.0,ACE Team
3,js41637,2013-09-08,227300.0,True,For a simple (it's actually not all that simpl...,2,Indie,Euro Truck Simulator 2,2013,19.99,227300.0,SCS Software
4,js41637,2013-09-08,227300.0,True,For a simple (it's actually not all that simpl...,2,Simulation,Euro Truck Simulator 2,2013,19.99,227300.0,SCS Software


In [10]:
df_developer.drop(columns=['item_id', 'id','review', 'genres', 'title', 'posted','price'],axis=1 ,inplace=True)

df_developer.head()

,user_id,recommend,sentiment_category,release_year,developer
0,76561197970982479,True,2,2009,Tripwire Interactive
1,76561197970982479,True,2,2009,ACE Team
2,76561197970982479,True,2,2009,ACE Team
3,js41637,True,2,2013,SCS Software
4,js41637,True,2,2013,SCS Software


In [11]:
df_developer = df_developer.merge(df_recs, left_on='user_id', right_on='user_id')
df_developer.head()

,user_id,recommend,sentiment_category,release_year,developer,rating,item_name
0,76561197970982479,True,2,2009,Tripwire Interactive,5,Killing Floor
1,76561197970982479,True,2,2009,Tripwire Interactive,5,Zeno Clash
2,76561197970982479,True,2,2009,Tripwire Interactive,5,Metro 2033
3,76561197970982479,True,2,2009,ACE Team,5,Killing Floor
4,76561197970982479,True,2,2009,ACE Team,5,Zeno Clash


In [12]:
df_developer.drop(columns=['item_name'],axis=1 ,inplace=True)
df_developer.head()

,user_id,recommend,sentiment_category,release_year,developer,rating
0,76561197970982479,True,2,2009,Tripwire Interactive,5
1,76561197970982479,True,2,2009,Tripwire Interactive,5
2,76561197970982479,True,2,2009,Tripwire Interactive,5
3,76561197970982479,True,2,2009,ACE Team,5
4,76561197970982479,True,2,2009,ACE Team,5


In [13]:
df_developer.duplicated().sum()

293581

In [14]:
df_developer.drop_duplicates(inplace=True)

In [15]:
def best_developer_year(year: int):

    # Filter the data for the given year
    df_year = df_developer[df_developer['release_year'] == year]

    # Calculate the number of recommendations for each developer
    dev_recommend_counts = df_year['developer'].value_counts()

    # Get the top 3 developers based on the number of recommendations
    top_devs_index = dev_recommend_counts.nlargest(3).index

    # Create a dictionary with the rankings and the corresponding developers
    rankings = ["1st place", "2nd place", "3rd place"]
    top_devs_dict = {rank: {'developer': dev, 'recommend': dev_recommend_counts[dev]} for rank, dev in zip(rankings, top_devs_index)}

    return top_devs_dict

In [16]:
best_developer_year(2015)

{'1st place': {'developer': 'Rockstar North', 'recommend': 518},
 '2nd place': {'developer': 'Kyle Seeley', 'recommend': 302},
 '3rd place': {'developer': 'Bethesda Game Studios', 'recommend': 285}}

In [20]:
pq.write_table(pa.Table.from_pandas(df_developer), '../data/df_developer.parquet')

## def developer_reviews_analysis(developer: str)
per developer, returns a dictionary containing the total count of reviews for each type: {'developer': [Negative = n, positive = p]}

In [21]:
def developer_reviews_analysis(developer):
  """
  Analyzes developer reviews and returns a dictionary with review counts.

  Args:
      developer: The name of the developer to analyze.

  Returns:
      A dictionary with the developer name as the key and a list containing 
      the count of negative and positive reviews as values.
  """

  # Merge reviews and games on item_id and developer
  # Specify how to handle differing column names if needed


  # Count reviews by sentiment category
  review_counts = df_developer['sentiment_category'].value_counts().to_dict()

  # Convert category counts to a list with desired format
  review_list = {
      'Negative' : review_counts.get(0, 0),  # Use integer 0 for negative category
      'Positive' : review_counts.get(2, 0),  # Use integer 2 for positive category
  }

  # Create the dictionary with developer name and review counts
  return {developer: review_list}

In [22]:
developer_reviews_analysis('Re-Logic')

{'Re-Logic': {'Negative': 13949, 'Positive': 46465}}

## def userforgenre(genre: str)
Returns the user with the most hours played in a given genre, playtime divided by year: {'genre': 'user_id', "Hours played": [{year: yyyy, hours: n},{year: yyyy, hours: n},{year: yyyy, hours: n}]}

In [23]:
df_genre = df_items.merge(df_games, left_on='item_name', right_on='title')
df_genre

,user_id,items_count,item_id,item_name,playtime_forever,genres,title,release_year,price,id,developer
0,76561197970982479,277,10,Counter-Strike,6.0,Action,Counter-Strike,2000,9.99,10.0,Valve
1,76561197970982479,277,20,Team Fortress Classic,0.0,Action,Team Fortress Classic,1999,4.99,20.0,Valve
2,76561197970982479,277,30,Day of Defeat,7.0,Action,Day of Defeat,2003,4.99,30.0,Valve
3,76561197970982479,277,40,Deathmatch Classic,0.0,Action,Deathmatch Classic,2001,4.99,40.0,Valve
4,76561197970982479,277,50,Half-Life: Opposing Force,0.0,Action,Half-Life: Opposing Force,1999,4.99,50.0,Gearbox Software
...,...,...,...,...,...,...,...,...,...,...,...
7631002,76561198329548331,7,388490,One Way To Die: Steam Edition,3.0,Adventure,One Way To Die: Steam Edition,2015,0.00,388490.0,CoaguCo Industries
7631003,76561198329548331,7,388490,One Way To Die: Steam Edition,3.0,Free to Play,One Way To Die: Steam Edition,2015,0.00,388490.0,CoaguCo Industries
7631004,76561198329548331,7,521570,You Have 10 Seconds 2,4.0,Casual,You Have 10 Seconds 2,2016,0.00,521570.0,Tamationgames
7631005,76561198329548331,7,521570,You Have 10 Seconds 2,4.0,Free to Play,You Have 10 Seconds 2,2016,0.00,521570.0,Tamationgames


In [24]:
df_genre.drop(columns=['items_count','title','id','item_name','developer','price'],axis=1 ,inplace=True)

df_genre

,user_id,item_id,playtime_forever,genres,release_year
0,76561197970982479,10,6.0,Action,2000
1,76561197970982479,20,0.0,Action,1999
2,76561197970982479,30,7.0,Action,2003
3,76561197970982479,40,0.0,Action,2001
4,76561197970982479,50,0.0,Action,1999
...,...,...,...,...,...
7631002,76561198329548331,388490,3.0,Adventure,2015
7631003,76561198329548331,388490,3.0,Free to Play,2015
7631004,76561198329548331,521570,4.0,Casual,2016
7631005,76561198329548331,521570,4.0,Free to Play,2016


In [25]:
print(df_genre.duplicated().sum())
df_genre.drop_duplicates(inplace=True)

10


In [26]:
# Group by user_id and count the occurrences of item_id
user_id_counts = df_genre.groupby('user_id')['item_id'].count().reset_index(name='count')
# Filter the dataframe to include only user_id's with count greater than a threshold (e.g., 1000)
filtered_user_ids = user_id_counts[user_id_counts['count'] > 300]['user_id']

# Filter df_genres based on the filtered user_ids
df_genre = df_genre[df_genre['user_id'].isin(filtered_user_ids)]


In [27]:
df_genre

,user_id,item_id,playtime_forever,genres,release_year
0,76561197970982479,10,6.0,Action,2000
1,76561197970982479,20,0.0,Action,1999
2,76561197970982479,30,7.0,Action,2003
3,76561197970982479,40,0.0,Action,2001
4,76561197970982479,50,0.0,Action,1999
...,...,...,...,...,...
7630987,76561198326700687,358390,0.0,Action,1996
7630988,76561198326700687,358390,0.0,Free to Play,1996
7630989,76561198326700687,521570,0.0,Casual,2016
7630990,76561198326700687,521570,0.0,Free to Play,2016


In [28]:
df_genre['playtime_hours'] = df_genre['playtime_forever'].apply(lambda x: x/60 if x != 0 else 0)
df_genre

,user_id,item_id,playtime_forever,genres,release_year,playtime_hours
0,76561197970982479,10,6.0,Action,2000,0.100000
1,76561197970982479,20,0.0,Action,1999,0.000000
2,76561197970982479,30,7.0,Action,2003,0.116667
3,76561197970982479,40,0.0,Action,2001,0.000000
4,76561197970982479,50,0.0,Action,1999,0.000000
...,...,...,...,...,...,...
7630987,76561198326700687,358390,0.0,Action,1996,0.000000
7630988,76561198326700687,358390,0.0,Free to Play,1996,0.000000
7630989,76561198326700687,521570,0.0,Casual,2016,0.000000
7630990,76561198326700687,521570,0.0,Free to Play,2016,0.000000


In [30]:
def UserForGenre(genre:str):
  # Filter data for the given genre
  genre_data = df_genre[df_genre['genres'].apply(lambda x: genre in x)]

  # Calculate total playtime per user per year (assuming playtime_forever in hours)
  user_year_playtime = (
      genre_data
      .groupby(['user_id', genre_data['release_year']])['playtime_hours']
      .sum()
      .reset_index()
  )

  # Group by user ID and sum playtime across years
  user_playtime_total = df_genre.groupby('user_id')['playtime_hours'].sum()

  # Find user with the most playtime
  top_user_id = user_playtime_total.idxmax()

  # Filter data for the top user
  top_user_data = user_year_playtime[user_year_playtime['user_id'] == top_user_id]

  # Prepare playtime details
  playtime_details = [
      {'year': row["release_year"], 'hours': round(row["playtime_hours"], 2)}
      for _, row in top_user_data.iterrows()
  ]

  # Return user details dictionary
  return {
      "genre": genre,
      "user_id": top_user_id,
      "Hours played": playtime_details
  }

In [31]:
UserForGenre('Adventure')

{'genre': 'Adventure',
 'user_id': 'REBAS_AS_F-T',
 'Hours played': [{'year': 2002, 'hours': 0.0},
  {'year': 2003, 'hours': 31.05},
  {'year': 2005, 'hours': 18.55},
  {'year': 2006, 'hours': 0.0},
  {'year': 2007, 'hours': 0.0},
  {'year': 2008, 'hours': 20.68},
  {'year': 2009, 'hours': 281.88},
  {'year': 2010, 'hours': 267.82},
  {'year': 2011, 'hours': 916.9},
  {'year': 2012, 'hours': 1797.15},
  {'year': 2013, 'hours': 3428.13},
  {'year': 2014, 'hours': 2582.68},
  {'year': 2015, 'hours': 9590.17},
  {'year': 2016, 'hours': 11928.88},
  {'year': 2017, 'hours': 543.67}]}

In [32]:
pq.write_table(pa.Table.from_pandas(df_genre), '../data/df_genre.parquet')

## def userdata(user_id: str)
per user returns a dictionary containing: {user id: amount of money spent, percentage of recommendation, number of items}

In [33]:
df_items['item_id'] = pd.to_numeric(df_items['item_id'])
df_userdata = df_items.merge(df_games, left_on='item_id', right_on='id')
df_userdata.head()

,user_id,items_count,item_id,item_name,playtime_forever,genres,title,release_year,price,id,developer
0,76561197970982479,277,10,Counter-Strike,6.0,Action,Counter-Strike,2000,9.99,10.0,Valve
1,76561197970982479,277,20,Team Fortress Classic,0.0,Action,Team Fortress Classic,1999,4.99,20.0,Valve
2,76561197970982479,277,30,Day of Defeat,7.0,Action,Day of Defeat,2003,4.99,30.0,Valve
3,76561197970982479,277,40,Deathmatch Classic,0.0,Action,Deathmatch Classic,2001,4.99,40.0,Valve
4,76561197970982479,277,50,Half-Life: Opposing Force,0.0,Action,Half-Life: Opposing Force,1999,4.99,50.0,Gearbox Software


In [34]:
df_userdata.drop(columns=['playtime_forever','title','id','item_name', 'release_year','developer','genres'],axis=1 ,inplace=True)

df_userdata.head()

,user_id,items_count,item_id,price
0,76561197970982479,277,10,9.99
1,76561197970982479,277,20,4.99
2,76561197970982479,277,30,4.99
3,76561197970982479,277,40,4.99
4,76561197970982479,277,50,4.99


In [35]:
df_userdata = df_userdata.merge(df_reviews, left_on='user_id', right_on='user_id')
df_userdata.head()

,user_id,items_count,item_id_x,price,posted,item_id_y,recommend,review,sentiment_category
0,76561197970982479,277,10,9.99,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,277,10,9.99,2011-07-15,22200.0,True,It's unique and worth a playthrough.,2
2,76561197970982479,277,10,9.99,2011-04-21,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,2
3,76561197970982479,277,20,4.99,2011-11-05,1250.0,True,Simple yet with great replayability. In my opi...,2
4,76561197970982479,277,20,4.99,2011-07-15,22200.0,True,It's unique and worth a playthrough.,2


In [36]:
df_userdata.drop(columns=['posted','item_id_y','review', 'sentiment_category'],axis=1 ,inplace=True)
df_userdata.head()

,user_id,items_count,item_id_x,price,recommend
0,76561197970982479,277,10,9.99,True
1,76561197970982479,277,10,9.99,True
2,76561197970982479,277,10,9.99,True
3,76561197970982479,277,20,4.99,True
4,76561197970982479,277,20,4.99,True


In [37]:
# change name of column 'item_id_x' in df_userdata  to item_id
df_userdata.rename(columns={"item_id_x": "item_id"}, inplace=True)
df_userdata.head()

,user_id,items_count,item_id,price,recommend
0,76561197970982479,277,10,9.99,True
1,76561197970982479,277,10,9.99,True
2,76561197970982479,277,10,9.99,True
3,76561197970982479,277,20,4.99,True
4,76561197970982479,277,20,4.99,True


In [38]:
df_userdata.drop(columns=['item_id'],axis=1 ,inplace=True)
df_userdata.head()

,user_id,items_count,price,recommend
0,76561197970982479,277,9.99,True
1,76561197970982479,277,9.99,True
2,76561197970982479,277,9.99,True
3,76561197970982479,277,4.99,True
4,76561197970982479,277,4.99,True


In [39]:
df_userdata.head()

,user_id,items_count,price,recommend
0,76561197970982479,277,9.99,True
1,76561197970982479,277,9.99,True
2,76561197970982479,277,9.99,True
3,76561197970982479,277,4.99,True
4,76561197970982479,277,4.99,True


In [40]:
df_userdata.duplicated().sum()


11606039

In [41]:
df_userdata.drop_duplicates(inplace=True)

In [43]:
pq.write_table(pa.Table.from_pandas(df_userdata), '../data/df_userdata.parquet')

In [44]:
def userdata(user_id: str):

    # Filter user_items by user_id
    user_items = df_userdata.loc[df_userdata['user_id'] == user_id]

    # Calculate money spent
    money_spent = user_items['price'].sum()

    # Get the number of items
    number_of_items = float(user_items['items_count'].unique()[0])

    # Filter and count only True values (recommendations)
    user_recommendations = user_items['recommend']
    recommend_rate = user_recommendations.where(user_recommendations == True).count() 

    # Calculate total items to avoid division by zero
    total_items = user_items['items_count'].sum()

    # Calculate recommendation rate (avoiding division by zero)
    recommend_rate = recommend_rate / total_items if total_items > 0 else 0

    user_data = {
        'user id': user_id,
        'money spent': round(money_spent, 2),
        'number of items': number_of_items,
        'recommend rate': round(recommend_rate,3)
    }
    return user_data

In [45]:
userdata('76561197970982479')

{'user id': '76561197970982479',
 'money spent': 294.32,
 'number of items': 277.0,
 'recommend rate': 0.004}

In [46]:
userdata('Derp-e')

{'user id': 'Derp-e',
 'money spent': 89.95,
 'number of items': 18.0,
 'recommend rate': 0.056}